In [1]:
# https://github.com/intel-isl/Open3D-ML
# http://www.open3d.org/docs/release/tutorial/geometry/
# https://analyticsindiamag.com/guide-to-open3d-an-open-source-modern-library-for-3d-data-processing/
# https://api-2d3d-cad.com/mesh/

In [5]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import copy
import os
import sys

lol = ''

clouds_path = ''
test_cloud_name = 'cloud_2_1620665787167437.pcd'
test_cloud_path = os.path.join(clouds_path, test_cloud_name)
test_cloud_path2 = os.path.join(clouds_path, 'cloud_2_1620725866961481.pcd')

In [3]:
o3d.visualization.draw_geometries([o3d.io.read_point_cloud(lol)])

In [4]:
test_cloud = o3d.io.read_point_cloud(test_cloud_path)

In [5]:
type(test_cloud)
print(np.asarray(test_cloud.points).shape)
o3d.visualization.draw_geometries([test_cloud])

(610227, 3)


In [6]:
print("Downsample the point cloud with a voxel of 0.05")
downpcd = test_cloud.voxel_down_sample(voxel_size=0.02)
# pcd.uniform_down_sample(every_k_points=5)
o3d.visualization.draw_geometries([downpcd])

Downsample the point cloud with a voxel of 0.05


In [7]:
bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(-2, 0.5, 1),
max_bound=(-1, 1, 3.5))
bbox_pc = downpcd.crop(bbox)
o3d.visualization.draw_geometries([bbox_pc])

In [111]:
aabb = bbox_pc.get_axis_aligned_bounding_box()
aabb.color = (1, 0, 0)
obb = bbox_pc.get_oriented_bounding_box()
obb.color = (0, 1, 0)
o3d.visualization.draw_geometries([test_cloud, bbox_pc, aabb, obb])

In [112]:
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(downpcd.cluster_dbscan(eps=0.05, min_points=10, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
downpcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([downpcd])

[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 74
point cloud has 74 clusters


In [125]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

print("Statistical oulier removal")
# cl, ind = downpcd.remove_statistical_outlier(nb_neighbors=50, std_ratio=1.0)
cl, ind = downpcd.remove_radius_outlier(nb_points=10, radius=0.05)
display_inlier_outlier(downpcd, ind)

Statistical oulier removal
Showing outliers (red) and inliers (gray): 


In [120]:
points = np.random.rand(10000, 3)
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(points)
o3d.visualization.draw_geometries([point_cloud])

In [126]:
print('voxelization')
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(downpcd,
                                                            voxel_size=0.01)
o3d.visualization.draw_geometries([voxel_grid])

voxelization


In [ ]:
def preprocess_pcl(pcl):
    print('pcl', np.asarray(test_cloud.points).shape)
    downpcl = pcl.voxel_down_sample(voxel_size=0.05)
    print('pcl downsampl', np.asarray(downpcl.points).shape)
    rm_pcl, ind = downpcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    # cl, ind = voxel_down_pcd.remove_radius_outlier(nb_points=16, radius=0.05)

In [ ]:
# -----------------------------------------------------------------------------------------

In [4]:
def display_inlier_outlier(pcl, ind):
    inlier_cloud = pcl.select_by_index(ind)
    outlier_cloud = pcl.select_by_index(ind, invert=True)
    # outlier_cloud.paint_uniform_color([1, 0, 0])
    # inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    return inlier_cloud, outlier_cloud

def preprocess_pcl(pcl):
    print('pcl', np.asarray(pcl.points).shape)
    downpcl = test_cloud.voxel_down_sample(voxel_size=0.03)
    # downpcl = pcl.uniform_down_sample(every_k_points=100)
    #downpcl = pcl.select_down_sample(indicies)
    print('pcl downsampl', np.asarray(downpcl.points).shape)
    rm_pcl, ind = downpcl.remove_statistical_outlier(nb_neighbors=10, std_ratio=2.0)
    # cl, ind = voxel_down_pcd.remove_radius_outlier(nb_points=16, radius=0.05)
    print('pcl rmsampl', np.asarray(rm_pcl.points).shape)
    inlier_cloud, outlier_cloud = display_inlier_outlier(downpcl, ind)
    return inlier_cloud

def get_ground_plane(pcl):
    plane_model, inliers = pcl.segment_plane(distance_threshold=0.15,ransac_n=3,num_iterations=1000)
    [a, b, c, d] = plane_model
    print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
    return inliers

def pcl_to_voxels(pcl):
    voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcl,
                                                                voxel_size=0.05)
    return voxel_grid

def pcl_clustering(pcl):
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(pcl.cluster_dbscan(eps=0.1, min_points=10, print_progress=True))

    max_label = labels.max()
    print(f"point cloud has {max_label + 1} clusters")
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcl.colors = o3d.utility.Vector3dVector(colors[:, :3])
    return pcl

In [97]:
test_cloud = o3d.io.read_point_cloud(test_cloud_path)
test_cloud2 = o3d.io.read_point_cloud(test_cloud_path2)
test_cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=16), fast_normal_computation=True)
prep_pcl = preprocess_pcl(test_cloud)
prep_pcl2 = preprocess_pcl(test_cloud2)

pcl (610227, 3)
pcl downsampl (39177, 3)
pcl rmsampl (38397, 3)
pcl (695577, 3)
pcl downsampl (39177, 3)
pcl rmsampl (38382, 3)


In [98]:
o3d.visualization.draw_geometries([test_cloud])

In [99]:
ground_plane_ind = get_ground_plane(prep_pcl)
gp_inl, gp_outl = display_inlier_outlier(prep_pcl, ground_plane_ind)
gp_inl.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([gp_inl, gp_outl])

Plane equation: 0.00x + 0.06y + 1.00z + -3.34 = 0


In [100]:
clust_pcl = pcl_clustering(gp_outl)
o3d.visualization.draw_geometries([clust_pcl])

[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 61
point cloud has 61 clusters


In [22]:
# -----------------------------------------------------------------

In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import copy
import os
import sys
lol = ''

In [2]:
clouds_path = ''
test_cloud_name = 'cloud_2_1620665787167437.pcd'
test_cloud_path = os.path.join(clouds_path, test_cloud_name)

print('load pcl')
pcl = o3d.io.read_point_cloud(lol,remove_nan_points = True, remove_infinite_points = True)
type(pcl)
print(np.asarray(pcl.points).shape)
o3d.visualization.draw_geometries([pcl])

load pcl
(705205, 3)


In [3]:
print("downsample the point cloud with a voxels")
downpcl = pcl.voxel_down_sample(voxel_size=0.03)
# downpcl = pcl.uniform_down_sample(every_k_points=15)
# downpcl = pcl.select_down_sample(indicies)
o3d.visualization.draw_geometries([downpcl])
print(np.asarray(downpcl.points).shape)

downsample the point cloud with a voxels
(46013, 3)


In [4]:
print('remove ground plane points')
ground_plane_model, ground_ind = downpcl.segment_plane(distance_threshold=0.2,ransac_n=3,num_iterations=1000)
[a, b, c, d] = ground_plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
gp_pcl = downpcl.select_by_index(ground_ind)
gp_pcl.paint_uniform_color([1, 0, 0])
pcl_without_gp = downpcl.select_by_index(ground_ind, invert=True)
o3d.visualization.draw_geometries([pcl_without_gp, gp_pcl])

gp_axis_bbox = gp_pcl.get_axis_aligned_bounding_box()
gp_or_bbox = gp_pcl.get_oriented_bounding_box()
gp_axis_bbox.color = (1, 0, 0)
gp_or_bbox.color = (0, 1, 0)
o3d.visualization.draw_geometries([downpcl, gp_axis_bbox, gp_or_bbox])

minb = gp_axis_bbox.get_min_bound()
maxb = gp_axis_bbox.get_max_bound()
maxb[2] = 4.0 # clip ground plane
bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=minb,
max_bound=maxb)
gp_ind = bbox.get_point_indices_within_bounding_box(downpcl.points)
gpbbox_inl = downpcl.select_by_index(gp_ind)
pcl_without_gp = downpcl.select_by_index(gp_ind, invert=True)
# gpbbox_outl.paint_uniform_color([1, 0, 0])
# inlier_cloud.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([pcl_without_gp])

remove ground plane points
Plane equation: -0.09x + -0.04y + 1.00z + -3.23 = 0


In [5]:
# print('select train points') # ???
# train_model, train_ind = pcl_without_gp.segment_plane(distance_threshold=0.2,ransac_n=3,num_iterations=1000)
# [a, b, c, d] = train_model
# print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
# train_pcl = pcl_without_gp.select_by_index(train_ind)
# train_pcl.paint_uniform_color([1, 0, 0])
# pcl_workspace = pcl_without_gp.select_by_index(train_ind, invert=True)
# o3d.visualization.draw_geometries([train_pcl, pcl_workspace])

# rmpcl, ind = train_pcl.remove_statistical_outlier(nb_neighbors=20,
# std_ratio=1.0)
# train_inlier_cloud = train_pcl.select_by_index(ind)
# train_outlier_cloud = train_pcl.select_by_index(ind, invert=True)
# train_outlier_cloud.paint_uniform_color([1, 0, 0])
# # inlier_cloud.paint_uniform_color([1, 0, 0])
# o3d.visualization.draw_geometries([train_inlier_cloud, train_outlier_cloud])
# train_pcl = train_inlier_cloud
# o3d.visualization.draw_geometries([train_pcl])

In [6]:
pcl_workspace = pcl_without_gp
print("remove noise from workspace")
rmpcl, work_ind = pcl_workspace.remove_statistical_outlier(nb_neighbors=10,
std_ratio=3.0)
# workspace, work_ind = pcl_workspace.remove_radius_outlier(nb_points=25, radius=0.1)
work_inlier_cloud = pcl_workspace.select_by_index(work_ind)
work_outlier_cloud = pcl_workspace.select_by_index(work_ind, invert=True)
work_outlier_cloud.paint_uniform_color([1, 0, 0])
# inlier_cloud.paint_uniform_color([1, 0, 0])
# pcl_workspace = work_inlier_cloud
respcl = work_inlier_cloud
o3d.visualization.draw_geometries([respcl])

remove noise from workspace


In [7]:
train_area = o3d.geometry.AxisAlignedBoundingBox(min_bound=(-2, -1.5, 1),
max_bound=(2, -0.0, 3))
# train_cloud = respcl.crop(train_area)
# bbox_train = train_cloud.get_axis_aligned_bounding_box()
# o3d.visualization.draw_geometries([train_area, respcl])
train_ind = train_area.get_point_indices_within_bounding_box(respcl.points)
train_pcl = respcl.select_by_index(train_ind)
o3d.visualization.draw_geometries([train_pcl, train_area])

danger_area = o3d.geometry.AxisAlignedBoundingBox(min_bound=(-2, -0.4, 1),
max_bound=(2, 1, 3))
# danger_cloud = respcl.crop(danger_area)
# bbox_danger = danger_cloud.get_axis_aligned_bounding_box()
danger_ind = danger_area.get_point_indices_within_bounding_box(respcl.points)
danger_pcl = respcl.select_by_index(danger_ind)
o3d.visualization.draw_geometries([danger_pcl, danger_area])

In [8]:
# rmpcl, rm_danger_ind = danger_pcl.remove_statistical_outlier(nb_neighbors=10,
# std_ratio=1.0)
mpcl, rm_danger_ind = danger_pcl.remove_radius_outlier(nb_points=20, radius=0.1)
result_pcl = danger_pcl.select_by_index(rm_danger_ind)
ppcl3 = danger_pcl.select_by_index(rm_danger_ind, invert=True)
ppcl3.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([result_pcl, ppcl3])

In [9]:
labels = np.array(result_pcl.cluster_dbscan(eps=0.1, min_points=10, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
result_pcl.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([result_pcl])

point cloud has 9 clusters


In [11]:
import numpy as np
%load model.py

In [12]:
from model import get_keras_pointnet

In [40]:
pointnet = get_keras_pointnet(4, 4096)
pointnet.load_weights('./pointnet_weights2.h5')

In [41]:
pointnet.summary()

Model: "pointnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 4096, 3)]    0                                            
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 4096, 32)     128         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_17 (BatchNo (None, 4096, 32)     128         conv1d_11[0][0]                  
__________________________________________________________________________________________________
activation_17 (Activation)      (None, 4096, 32)     0           batch_normalization_17[0][0]     
___________________________________________________________________________________________

In [42]:
classes = ['human', 'wear', 'limb', 'other']
instances = np.unique(labels)[1:]
bboxes_pcs = []
output = []
for instance in instances:
    pc = o3d.geometry.PointCloud() 
    pc.points = o3d.utility.Vector3dVector(np.asarray(result_pcl.points)[np.where(labels == instance)])
    pc_bbox = pc.get_axis_aligned_bounding_box() 
    bboxes_pcs.append(pc_bbox)
    bbox_pcl = result_pcl.crop(pc_bbox)
    bbox_pcl_arr = np.asarray(bbox_pcl.points)
    bbox_pcl_arr = bbox_pcl_arr[np.newaxis, :]
    pred = pointnet.predict(bbox_pcl_arr)
    pred = pred.argmax()
    print(pred)
    center = bbox_pcl.get_center()
    min_bound = bbox_pcl.get_min_bound()
    max_bound = bbox_pcl.get_max_bound()
    
    # o3d.visualization.draw_geometries([pc_bbox, result_pcl])
    obj = {'object':classes[pred], 
           'geometry':{'position': {'x':center[0], 'y':center[1], 'z':center[2]},
                       'rotation': {'x':0, 'y':0, 'z':0},
                       'dimensions': {
                           'x':abs(max_bound[0]-min_bound[0]), 
                           'y':abs(max_bound[1]-min_bound[1]), 
                           'z':abs(max_bound[2]-min_bound[2])}},
           'door': 'unknown'}
    output.append(obj)

out_json = {'figures':obj}
# o3d.visualization.draw_geometries([*bboxes_pcs, result_pcl])

WARNING - 2021-05-23 09:48:56,980 - functional - Model was constructed with shape (None, 4096, 3) for input Tensor("input_2:0", shape=(None, 4096, 3), dtype=float32), but it was called on an input with incompatible shape (None, 2145, 3).
WARNING - 2021-05-23 09:48:57,249 - functional - Model was constructed with shape (None, 4096, 3) for input Tensor("input_2:0", shape=(None, 4096, 3), dtype=float32), but it was called on an input with incompatible shape (None, 127, 3).
0
0
0
3
0
0
0
0
0


In [43]:
import json

with open('out.json', 'w') as fp:
    json.dump(out_json, fp)